# Building a ReAct Agent

Build an LLM agent that controls Grasshopper using the **ReAct** (Reasoning + Acting) framework.

**What you'll build:**
- An `Agent` class that maintains conversation history
- Connection to Grasshopper via MCP (Model Context Protocol)  
- A ReAct loop: Thought → Action → Observation → repeat

---

## Part 1: Setup

In [6]:
import re, json, os
from dotenv import load_dotenv
import httpx

# Provider imports - both are available
from groq import Groq

# For Gemini support (install with: pip install google-genai)
try:
    from google import genai
    from google.genai import types
    GEMINI_AVAILABLE = True
except ImportError:
    GEMINI_AVAILABLE = False
    print("Note: google-genai not installed. Run 'pip install google-genai' to use Gemini.")

load_dotenv()  # Load API keys from .env file

True

In [8]:
# =============================================================================
# PROVIDER TOGGLE - Change this to switch between Groq and Gemini
# =============================================================================
USE_GEMINI = False  # Set to True to use Gemini 2.5 Flash, False for Groq (Llama 3.3)

# API Keys:
# - Groq (FREE): https://console.groq.com/keys
# - Gemini (FREE): https://aistudio.google.com/apikey

if USE_GEMINI:
    if not GEMINI_AVAILABLE:
        raise ImportError("google-genai not installed! Run: pip install google-genai")
    
    API_KEY = os.environ.get("GOOGLE_API_KEY")
    if not API_KEY:
        raise ValueError("GOOGLE_API_KEY not found! Add it to your .env file.")
    
    client = genai.Client(api_key=API_KEY)
    MODEL_NAME = "gemini-2.5-flash"
    print(f"Using Gemini: {MODEL_NAME}")
    
    # Quick test
    response = client.models.generate_content(
        model=MODEL_NAME,
        contents="Tell me a haiku about Behavioral robotics"
    )
    print(response.text)
else:
    API_KEY = os.environ.get("GROQ_API_KEY")
    if not API_KEY:
        raise ValueError("GROQ_API_KEY not found! Copy .env.example to .env and add your key.")
    
    client = Groq(api_key=API_KEY)
    MODEL_NAME = "llama-3.3-70b-versatile"
    print(f"Using Groq: {MODEL_NAME}")
    
    # Quick test
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": "Tell me a haiku about Behavioral robotics"}],
        temperature=1
    )
    print(response.choices[0].message.content)

Using Groq: llama-3.3-70b-versatile
Metal minds learn slow
Actions born of code and trial
Robots think, adapt rise


### LLM Wrapper

Wrap the API call in a function. To switch providers (OpenAI, Anthropic, etc.), just change this function.

In [ ]:
def call_llm(messages: list[dict], temperature: float = 0) -> str:
    """Call the LLM with a list of messages, return response text.
    
    Works with both Groq and Gemini based on USE_GEMINI toggle.
    """
    if USE_GEMINI:
        # Convert messages to Gemini format
        gemini_contents = []
        system_instruction = None
        
        for msg in messages:
            role = msg["role"]
            content = msg["content"]
            
            if role == "system":
                system_instruction = content
            elif role == "user":
                gemini_contents.append(types.Content(role="user", parts=[types.Part.from_text(text=content)]))
            elif role == "assistant":
                gemini_contents.append(types.Content(role="model", parts=[types.Part.from_text(text=content)]))
        
        # Configure generation
        config = types.GenerateContentConfig(
            temperature=temperature,
            max_output_tokens=4096,
            system_instruction=system_instruction,
        )
        
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=gemini_contents,
            config=config
        )
        return response.text
    else:
        # Groq (OpenAI-compatible)
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            temperature=temperature,
            max_tokens=4096
        )
        return response.choices[0].message.content

---

## Part 2: Agent Class

An agent is just:
1. A **system prompt** that defines behavior
2. A **message history** (list of messages)
3. A method to **call the LLM** and track the conversation

In [ ]:
class Agent:

## Part 2: Agent Class

        """Initialize with optional system prompt."""
        # TODO: Store system_prompt as self.system_prompt
        # TODO: Initialize self.messages = []
        # TODO: If system_prompt provided, append {"role": "system", "content": system_prompt}

    def __init__(self, systetm_prompt: str = "")
        self.system_prompt = system_prompt
        self.messages = []

        if system_system:
            self.messages.append({"role": "system", "content": system_prompt})



    def __call__(self, user_message: str) -> str:
        """Send a message and get a response."""
        # TODO: Append user message to self.messages
        # TODO: Call call_llm(self.messages) to get response
        # TODO: Append assistant response to self.messages
        # TODO: Return the response
        
        self.messages.append({"role": "system", "content": user_message})

        response = call_llm(self.messages)

        self.messages.append({"role": "system", "content": response})

        return response


    def reset(self):
        """Clear history but keep system prompt."""
        # TODO: Reset self.messages to []
        # TODO: Re-add system prompt if it exists
        self.message[]
        if self.system_prompt:
            self.message.append({"role": "system", "content": self.system_prompt}) 

IndentationError: unindent does not match any outer indentation level (<string>, line 18)

In [ ]:
# Test your Agent - should remember the conversation
agent = Agent("You are helpful. Be concise.")
print(agent("What is 2+2?"))
print(agent("What did I just ask?"))  # Tests memory

---

## Part 3: MCP Tools (Grasshopper)

Tools we'll use:
- `List_Python_Scripts` - Find script components
- `Get_Python_Script` - Read script code
- `Edit_Python_Script` - Write/modify code  
- `Get_Python_Script_Errors` - Check for errors

Make sure Grasshopper is running with MCP server active!

In [ ]:
MCP_URL = "http://127.0.0.1:8089/mcp"

def call_mcp_tool(tool_name: str, arguments: dict = None) -> dict:
    """Call a tool on the Grasshopper MCP server using SSE streaming."""
    payload = {
        "jsonrpc": "2.0", "id": 1,
        "method": "tools/call",
        "params": {"name": tool_name, "arguments": arguments or {}}
    }
    try:
        with httpx.Client(timeout=30) as client:
            with client.stream("POST", MCP_URL, json=payload) as response:
                for line in response.iter_lines():
                    if line.startswith("data: "):
                        data = json.loads(line[6:])
                        return data.get("result", data)
        return {"error": "No response"}
    except Exception as e:
        return {"error": str(e)}

In [ ]:
def get_available_tools() -> list:
    """Fetch available tools from the MCP server."""
    payload = {"jsonrpc": "2.0", "id": 1, "method": "tools/list", "params": {}}
    try:
        with httpx.Client(timeout=30) as client:
            with client.stream("POST", MCP_URL, json=payload) as response:
                for line in response.iter_lines():
                    if line.startswith("data: "):
                        data = json.loads(line[6:])
                        if "result" in data and "tools" in data["result"]:
                            return data["result"]["tools"]
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []

# Fetch tools and filter to the ones we need
ALL_TOOLS = get_available_tools()
TUTORIAL_TOOLS = [t for t in ALL_TOOLS if t["name"] in [
    "List_Python_Scripts", "Get_Python_Script", "Edit_Python_Script", "Get_Python_Script_Errors"
]]
print(f"Found {len(TUTORIAL_TOOLS)} tools: {[t['name'] for t in TUTORIAL_TOOLS]}")

In [ ]:
# Test: List Python scripts on the Grasshopper canvas
call_mcp_tool("List_Python_Scripts")

---

## Part 4: The ReAct Prompt

**ReAct** = Reasoning + Acting. The LLM follows a loop:

1. **Thought** - Reason about what to do
2. **Action** - Call a tool, then PAUSE
3. **Observation** - Receive tool result
4. **Repeat** until ready to give final **Answer**

The prompt tells the LLM exactly how to behave:

In [ ]:
def build_react_prompt(tools: list) -> str:
    """Build the ReAct system prompt with tool descriptions."""
    tool_docs = []
    for tool in tools:
        name = tool["name"]
        desc = tool["description"].split('.')[0]
        params = list(tool.get("inputSchema", {}).get("properties", {}).keys())
        tool_docs.append(f"- {name}: {desc}. Params: {params or 'none'}")
    
    return f"""You are a Grasshopper Python scripting assistant.

<rules>
- Output geometry by assigning to 'a': a = points
- Import Rhino.Geometry as rg
- Always check for errors after editing code
</rules>

<tools>
{chr(10).join(tool_docs)}
</tools>

<workflow>
Thought: reason about what to do
Action: ToolName {{"param": "value"}}
PAUSE
(wait for Observation)
Answer: when done
</workflow>

<example>
User: Create 10 points in a spiral

Thought: Find the script component.
Action: List_Python_Scripts
PAUSE

Observation: [{{"id": "abc-123", "name": "Task"}}]

Thought: Write spiral code.
Action: Edit_Python_Script {{"componentId": "abc-123", "code": "import Rhino.Geometry as rg\\nimport math\\npoints = [rg.Point3d(math.cos(i)*i, math.sin(i)*i, 0) for i in range(10)]\\na = points"}}
PAUSE

Observation: Script updated

Thought: Check errors.
Action: Get_Python_Script_Errors {{"componentId": "abc-123"}}
PAUSE

Observation: No errors

Answer: Created spiral of 10 points.
</example>
"""

REACT_PROMPT = build_react_prompt(TUTORIAL_TOOLS)
print(f"Prompt length: {len(REACT_PROMPT)} chars")

### Why This Prompt Works

Notice the key elements:
1. **Role definition** - "You are a Grasshopper scripting assistant"
2. **Loop structure** - Explicit Thought → Action → PAUSE → Observation cycle
3. **Tool descriptions** - Exact syntax with examples
4. **Workflow rules** - "ALWAYS check for errors"
5. **Example session** - Shows the expected format

The LLM follows this because it's trained to follow instructions.
The more specific and structured your prompt, the more reliable the behavior.

---

## Part 5: The Execution Loop

Now we need code to:
1. Parse actions from LLM output
2. Execute the tool
3. Feed the result back as an Observation

In [ ]:
# Regex to match: Action: ToolName {"param": "value"}
ACTION_PATTERN = re.compile(r'^Action:\s*(\w+)\s*(\{.*\})?$', re.MULTILINE)

def parse_action(response: str) -> tuple[str, dict] | None:
    """Parse an action from LLM response. Returns (tool_name, args) or None."""
    # TODO: Use ACTION_PATTERN.search(response) to find a match
    # TODO: If match: extract tool_name from group(1), args JSON from group(2)
    # TODO: Parse args with json.loads() if present, else empty dict
    # TODO: Return (tool_name, args) or None if no match
    pass

In [ ]:
def execute_action(tool_name: str, args: dict) -> str:
    """Execute an MCP tool and return the result as a string."""
    # TODO: Call call_mcp_tool(tool_name, args)
    # TODO: If result has "error" key, return f"Error: {result['error']}"
    # TODO: Otherwise return json.dumps(result, indent=2)
    pass

In [ ]:
def query(question: str, max_turns: int = 10, verbose: bool = True) -> str:
    """Run the ReAct loop until we get an Answer."""
    # TODO: Create Agent with REACT_PROMPT
    # TODO: Set next_prompt = question
    # TODO: Loop up to max_turns:
    #   - Get response from agent(next_prompt)
    #   - If verbose: print turn number and response
    #   - If "Answer:" in response (without "PAUSE"): extract and return the answer
    #   - Parse action with parse_action(response)
    #   - If action found: execute it, set next_prompt = f"Observation: {result}"
    #   - If no action: set next_prompt = "Continue with an Action or provide your Answer."
    # TODO: Return "Max turns reached." if loop ends
    pass

---

## Part 6: Run It!

Make sure Grasshopper is open with `examples/task_template.gh` loaded.

In [ ]:
# Try different prompts!
query("Create a spiral staircase with steps")

---

## Summary

You built a ReAct agent that:
- Maintains conversation history
- Follows the Thought → Action → Observation loop  
- Connects to real tools via MCP

**Learn more:**
- [ReAct Paper (Yao et al., 2022)](https://arxiv.org/abs/2210.03629)
- [DeepLearning.AI Agentic AI Course](https://www.deeplearning.ai/courses/agentic-ai/)

---

## Bonus: Pretty Output

A nicer version using Rich library with color-coded panels.

In [ ]:
from rich.console import Console
from rich.panel import Panel
from rich.syntax import Syntax

console = Console()

def extract_thought(response: str) -> str:
    lines = response.split('\n')
    thought_lines = []
    in_thought = False
    for line in lines:
        if line.strip().startswith('Thought:'):
            in_thought = True
            thought_lines.append(line.replace('Thought:', '').strip())
        elif in_thought and (line.strip().startswith('Action:') or line.strip() == 'PAUSE'):
            break
        elif in_thought:
            thought_lines.append(line.strip())
    return ' '.join(thought_lines).strip()

def query_pretty(question: str, max_turns: int = 10) -> str:
    agent = Agent(REACT_PROMPT)
    next_prompt = question
    console.print(Panel(question, title="[bold white]Request[/bold white]", border_style="white"))

    for turn in range(max_turns):
        console.print(f"\n[bold cyan]━━━ Turn {turn + 1} ━━━[/bold cyan]")
        response = agent(next_prompt)
        
        thought = extract_thought(response)
        if thought:
            console.print(Panel(thought, title="[bold]💭 Thought[/bold]", border_style="blue"))

        if "Answer:" in response and "PAUSE" not in response:
            final_answer = response[response.find("Answer:") + 7:].strip()
            console.print(Panel(final_answer, title="[bold]✅ Answer[/bold]", border_style="green"))
            return final_answer

        action_result = parse_action(response)
        if action_result:
            tool_name, args = action_result
            console.print(Panel(f"[yellow]{tool_name}[/yellow] {json.dumps(args)}", title="[bold]⚡ Action[/bold]", border_style="green"))
            observation = execute_action(tool_name, args)
            console.print(Panel(Syntax(observation[:500], "json", theme="monokai"), title="[bold]👁 Observation[/bold]", border_style="yellow"))
            next_prompt = f"Observation: {observation}"
        else:
            next_prompt = "Continue with an Action or provide your Answer."

    return "Max turns reached."

In [ ]:
query_pretty("Create a 5x5 grid of points")